# Chapter 6

## Gaussian mixture pipeline

Use Machine Learning Methods to cluster cars and their CO2 emission. 
Dataset by Kaggle. More information can be found [here](https://www.kaggle.com/debajyotipodder/).

In [1]:
from pyspark.sql import SparkSession 

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Intro") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/12 17:56:00 WARN Utils: Your hostname, Gabriels-MacBook-Pro.local, resolves to a loopback address: 127.0.0.1; using 10.40.1.32 instead (on interface en0)
25/10/12 17:56:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/12 17:56:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.sql.types import StructField, StructType, StringType, DoubleType

custom_schema = StructType([
    StructField("Make", StringType(), True),
    StructField("Model", StringType(), True),
    StructField("Vehicle Class", StringType(), True),
    StructField("Cylinders", DoubleType(), True),
    StructField("Transmission", StringType(), True),
    StructField("Fuel Type", StringType(), True),
    StructField("Fuel Consumption City (L/100 km)", DoubleType(), True),
    StructField("Fuel Consumption Hwy (L/100 km)", DoubleType(), True),
    StructField("Fuel Consumption Comb (L/100 km)", DoubleType(), True),
    StructField("Fuel Consumption Comb (mpg)", DoubleType(), True),
    StructField("CO2", DoubleType(), True)
])

In [6]:
co2_data = spark.read.format("csv")\
    .schema(custom_schema) \
    .option("header", True) \
    .load("./static/CO2_Emissions_Canada.csv")

In [7]:
co2_data.take(2)

25/10/12 17:57:05 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 12, schema size: 11
CSV file: file:///Users/gafnts/Documents/Github/ml-with-spark/ml-with-spark/static/CO2_Emissions_Canada.csv


[Row(Make='ACURA', Model='ILX', Vehicle Class='COMPACT', Cylinders=2.0, Transmission='4', Fuel Type='AS5', Fuel Consumption City (L/100 km)=None, Fuel Consumption Hwy (L/100 km)=9.9, Fuel Consumption Comb (L/100 km)=6.7, Fuel Consumption Comb (mpg)=8.5, CO2=33.0),
 Row(Make='ACURA', Model='ILX', Vehicle Class='COMPACT', Cylinders=2.4, Transmission='4', Fuel Type='M6', Fuel Consumption City (L/100 km)=None, Fuel Consumption Hwy (L/100 km)=11.2, Fuel Consumption Comb (L/100 km)=7.7, Fuel Consumption Comb (mpg)=9.6, CO2=29.0)]

In [8]:
co2_data = co2_data.fillna(0.0)

In [9]:
co2_data.printSchema()

root
 |-- Make: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Vehicle Class: string (nullable = true)
 |-- Cylinders: double (nullable = false)
 |-- Transmission: string (nullable = true)
 |-- Fuel Type: string (nullable = true)
 |-- Fuel Consumption City (L/100 km): double (nullable = false)
 |-- Fuel Consumption Hwy (L/100 km): double (nullable = false)
 |-- Fuel Consumption Comb (L/100 km): double (nullable = false)
 |-- Fuel Consumption Comb (mpg): double (nullable = false)
 |-- CO2: double (nullable = false)



In [10]:
co2_data.take(2)

25/10/12 17:57:14 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 12, schema size: 11
CSV file: file:///Users/gafnts/Documents/Github/ml-with-spark/ml-with-spark/static/CO2_Emissions_Canada.csv


[Row(Make='ACURA', Model='ILX', Vehicle Class='COMPACT', Cylinders=2.0, Transmission='4', Fuel Type='AS5', Fuel Consumption City (L/100 km)=0.0, Fuel Consumption Hwy (L/100 km)=9.9, Fuel Consumption Comb (L/100 km)=6.7, Fuel Consumption Comb (mpg)=8.5, CO2=33.0),
 Row(Make='ACURA', Model='ILX', Vehicle Class='COMPACT', Cylinders=2.4, Transmission='4', Fuel Type='M6', Fuel Consumption City (L/100 km)=0.0, Fuel Consumption Hwy (L/100 km)=11.2, Fuel Consumption Comb (L/100 km)=7.7, Fuel Consumption Comb (mpg)=9.6, CO2=29.0)]

# Build Hasher

Turn the feature columns into one indexed column:

In [ ]:
from pyspark.ml.feature import FeatureHasher

cols_only_continues = [
        "Fuel Consumption City (L/100 km)", 
        "Fuel Consumption Hwy (L/100 km)",
        "Fuel Consumption Comb (L/100 km)",
]

hasher = FeatureHasher(
    outputCol="hashed_features", inputCols=cols_only_continues
)

# Build Selector

In [14]:
from pyspark.ml.feature import UnivariateFeatureSelector

selector = UnivariateFeatureSelector(
    outputCol="selectedFeatures", featuresCol="hashed_features", labelCol="CO2"
)

selector.setFeatureType("continuous")
selector.setLabelType("continuous")

UnivariateFeatureSelector_5912b96f9673

# Create GaussianMixture

In [ ]:
from pyspark.ml.clustering import GaussianMixture

gm = GaussianMixture(k=42, tol=0.01, seed=10, featuresCol="selectedFeatures", maxIter=100)

# Constructing - The Pipeline API

In [16]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[hasher, selector, gm])

model = pipeline.fit(co2_data)

25/10/12 18:14:49 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Fuel Type, Fuel Consumption City (L/100 km), Fuel Consumption Hwy (L/100 km), Fuel Consumption Comb (mpg)
 Schema: Fuel Consumption City (L/100 km), Fuel Consumption Hwy (L/100 km), Fuel Consumption Comb (L/100 km), CO2
Expected: Fuel Consumption City (L/100 km) but found: Fuel Type
CSV file: file:///Users/gafnts/Documents/Github/ml-with-spark/ml-with-spark/static/CO2_Emissions_Canada.csv
25/10/12 18:14:49 WARN DAGScheduler: Broadcasting large task binary with size 2.0 MiB
25/10/12 18:14:49 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Fuel Type, Fuel Consumption City (L/100 km), Fuel Consumption Hwy (L/100 km), Fuel Consumption Comb (mpg)
 Schema: Fuel Consumption City (L/100 km), Fuel Consumption Hwy (L/100 km), Fuel Consumption Comb (L/100 km), CO2
Expected: Fuel Consumption City (L/100 km) but found: Fuel Type
CSV file: file:///Users/gafnts/Documents/Github/ml-w

In [17]:
transformed_by_pipeline = model.transform(co2_data)

In [18]:
transformed_by_pipeline.printSchema()

root
 |-- Make: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Vehicle Class: string (nullable = true)
 |-- Cylinders: double (nullable = false)
 |-- Transmission: string (nullable = true)
 |-- Fuel Type: string (nullable = true)
 |-- Fuel Consumption City (L/100 km): double (nullable = false)
 |-- Fuel Consumption Hwy (L/100 km): double (nullable = false)
 |-- Fuel Consumption Comb (L/100 km): double (nullable = false)
 |-- Fuel Consumption Comb (mpg): double (nullable = false)
 |-- CO2: double (nullable = false)
 |-- hashed_features: vector (nullable = true)
 |-- selectedFeatures: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: integer (nullable = false)



# Persisting the pipeline to disk

In [ ]:
path_model_with_pip = "./tmp/pip_model"
model.write().overwrite().save(path_model_with_pip)